In [1]:
# load and reform xarray
import numpy as np
import xarray as xr

# Lade die .nc Datei
file_path = 'N:/gebhyd/3_Hyv/Diplomanden/2_Running/L_Nuesch/SWE_envidat/TI_1998_2022_LNU1.nc'
SWE_envidat = xr.open_dataset(file_path)

SWE_envidat = SWE_envidat.rename({'xx': 'lon', 'yy': 'lat'})

# Angenommen, dein Dataset heißt SWE_envidat
# Extrahiere die Koordinaten
lon = SWE_envidat.coords['lon'].values
lat = SWE_envidat.coords['lat'].values

# Erstelle die station_ids im Format "lon_lat"
station_ids = [f"{lon_val}_{lat_val}" for lat_val in lat for lon_val in lon]

# Konvertiere die Liste in ein eindimensionales NumPy-Array
station_ids_array = np.array(station_ids)

# Füge die station_id als eindimensionale Koordinate hinzu
SWE_envidat = SWE_envidat.assign_coords(station_id=('station_id', station_ids_array))


SWE_envidat = SWE_envidat.drop_vars(['dem', 'open', 'forest', 'romc_all'])
# Variable 'swee_all' in 'swe' umbenennen
SWE_envidat = SWE_envidat.rename({'swee_all': 'swe'})
# Ausgabe des aktualisierten Datasets
print(SWE_envidat)


<xarray.Dataset>
Dimensions:          (lat: 272, lon: 365, station_id: 99280, time: 8704)
Coordinates:
  * lon              (lon) float64 4.74e+05 4.75e+05 ... 8.37e+05 8.38e+05
  * lat              (lat) float64 5.3e+04 5.4e+04 5.5e+04 ... 3.23e+05 3.24e+05
  * time             (time) datetime64[ns] 1998-09-01 1998-09-02 ... 2022-06-30
  * station_id       (station_id) <U17 '474000.0_53000.0' ... '838000.0_324000.0'
Data variables:
    Projection_Type  float64 ...
    swe              (time, lat, lon) float64 ...
Attributes:
    date_created:                  09-Jun-2023 16:30:30
    creator_name:                  Rebecca Mott
    project:                       FSM2
    geospatial_lat_min:            []
    geospatial_lat_max:            []
    geospatial_lon_min:            []
    geospatial_lon_max:            []
    geospatial_lat_units:          m
    geospatial_lon_units:          m
    geospatial_vertical_min:       []
    geospatial_vertical_max:       []
    geospatial_vertica

In [2]:
# Schritt 1: Lösche die bestehende station_id, falls vorhanden
if 'station_id' in SWE_envidat.coords:
    SWE_envidat = SWE_envidat.drop_vars('station_id')

In [3]:
# Schritt 2: Längen und Breiten für jede Station in einer flachen Liste erstellen
flat_lon = SWE_envidat.lon.values.flatten()
flat_lat = SWE_envidat.lat.values.flatten()



In [4]:
# Schritt 3: Erstelle die neue Station_ID als Kombination von lon und lat
station_ids = [f"{lon}_{lat}" for lon, lat in zip(flat_lon, flat_lat)]



In [5]:
# Schritt 4: Umstrukturieren des Datasets auf die Dimensionen 'Station_ID' und 'time'
SWE_envidat_stacked = SWE_envidat.stack(Station_ID=("lat", "lon"))



In [6]:
print(SWE_envidat_stacked)

<xarray.Dataset>
Dimensions:          (Station_ID: 99280, time: 8704)
Coordinates:
  * time             (time) datetime64[ns] 1998-09-01 1998-09-02 ... 2022-06-30
  * Station_ID       (Station_ID) MultiIndex
  - lat              (Station_ID) float64 5.3e+04 5.3e+04 ... 3.24e+05 3.24e+05
  - lon              (Station_ID) float64 4.74e+05 4.75e+05 ... 8.38e+05
Data variables:
    Projection_Type  float64 ...
    swe              (time, Station_ID) float64 -9.999e+03 ... -9.999e+03
Attributes:
    date_created:                  09-Jun-2023 16:30:30
    creator_name:                  Rebecca Mott
    project:                       FSM2
    geospatial_lat_min:            []
    geospatial_lat_max:            []
    geospatial_lon_min:            []
    geospatial_lon_max:            []
    geospatial_lat_units:          m
    geospatial_lon_units:          m
    geospatial_vertical_min:       []
    geospatial_vertical_max:       []
    geospatial_vertical_units:     m
    geospatial_vertic

In [8]:
# Schritt 5: Konvertiere MultiIndex zurück in einen Dataset mit den neuen Koordinaten
SWE_envidat_stacked = SWE_envidat_stacked.reset_index('Station_ID')



In [9]:
print(SWE_envidat_stacked)

<xarray.Dataset>
Dimensions:          (Station_ID: 99280, time: 8704)
Coordinates:
  * time             (time) datetime64[ns] 1998-09-01 1998-09-02 ... 2022-06-30
    lat              (Station_ID) float64 5.3e+04 5.3e+04 ... 3.24e+05 3.24e+05
    lon              (Station_ID) float64 4.74e+05 4.75e+05 ... 8.38e+05
Dimensions without coordinates: Station_ID
Data variables:
    Projection_Type  float64 ...
    swe              (time, Station_ID) float64 -9.999e+03 ... -9.999e+03
Attributes:
    date_created:                  09-Jun-2023 16:30:30
    creator_name:                  Rebecca Mott
    project:                       FSM2
    geospatial_lat_min:            []
    geospatial_lat_max:            []
    geospatial_lon_min:            []
    geospatial_lon_max:            []
    geospatial_lat_units:          m
    geospatial_lon_units:          m
    geospatial_vertical_min:       []
    geospatial_vertical_max:       []
    geospatial_vertical_units:     m
    geospatial_vertical

In [11]:
# Schritt 6: Erstelle die neue Station_ID als Kombination aus lat und lon
SWE_envidat_stacked['Station_ID'] = [f"{lon}_{lat}" for lon, lat in zip(SWE_envidat_stacked['lon'].values, SWE_envidat_stacked['lat'].values)]

# Schritt 7: Überprüfe das neu strukturierte Dataset
print(SWE_envidat_stacked)

<xarray.Dataset>
Dimensions:          (Station_ID: 99280, time: 8704)
Coordinates:
  * time             (time) datetime64[ns] 1998-09-01 1998-09-02 ... 2022-06-30
    lat              (Station_ID) float64 5.3e+04 5.3e+04 ... 3.24e+05 3.24e+05
    lon              (Station_ID) float64 4.74e+05 4.75e+05 ... 8.38e+05
  * Station_ID       (Station_ID) <U17 '474000.0_53000.0' ... '838000.0_324000.0'
Data variables:
    Projection_Type  float64 ...
    swe              (time, Station_ID) float64 -9.999e+03 ... -9.999e+03
Attributes:
    date_created:                  09-Jun-2023 16:30:30
    creator_name:                  Rebecca Mott
    project:                       FSM2
    geospatial_lat_min:            []
    geospatial_lat_max:            []
    geospatial_lon_min:            []
    geospatial_lon_max:            []
    geospatial_lat_units:          m
    geospatial_lon_units:          m
    geospatial_vertical_min:       []
    geospatial_vertical_max:       []
    geospatial_vertic

In [13]:
# store the reformed xarray
SWE_envidat_stacked.to_netcdf('../CH_data/CH_input_data/SWE_data.nc')